In [1]:
import os
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Đường dẫn dữ liệu và file CSV chứa thông tin nhãn
DATA_DIR = "data"  # Thư mục chính chứa train, test, valid
CSV_FILE = "FruitDataFrame.csv"
OUTPUT_DIR = "Images_QA"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Đọc file CSV
df = pd.read_csv(CSV_FILE)

# Danh sách lưu dữ liệu cho tokenizer
questions, answers, image_features = [], [], []
image_paths_list = []  # Danh sách lưu image path cho mỗi QA pair

# Lấy danh sách các loại trái cây (để dùng cho one-hot encoding và mở rộng từ vựng)
fruit_classes = sorted(df['fruit_type'].unique())

# Duyệt qua từng dòng của DataFrame
for idx, row in df.iterrows():
    filename = row["filename"]
    fruit_type_gt = row["fruit_type"]
    count_gt = row["count"]
    subset = row["subset"]
    
    # Xác định đường dẫn ảnh dựa theo subset và fruit_type
    image_path = os.path.join(DATA_DIR, subset, fruit_type_gt, "images", filename)
    if not os.path.exists(image_path):
        continue

    # (Nếu cần) Load ảnh để trích đặc trưng bằng CNN hay các xử lý tiền xử lý khác
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Tạo vector đặc trưng đơn giản: one-hot của loại trái cây cộng với số lượng
    fruit_one_hot = np.zeros(len(fruit_classes))
    fruit_index = fruit_classes.index(fruit_type_gt)
    fruit_one_hot[fruit_index] = 1
    feature_vector = np.concatenate([fruit_one_hot, [float(count_gt)]])

    # Sinh cặp QA sử dụng thông tin từ file CSV
    qa_pairs = [
        # Câu hỏi về số lượng
        ("Trong ảnh số lượng trái cây là bao nhiêu?", f"<start> ảnh có tổng cộng {count_gt} trái cây <end>"),
        ("Ảnh có bao nhiêu trái cây?", f"<start> ảnh có {count_gt} trái cây <end>"),
        ("Có bao nhiêu trái cây trong ảnh?", f"<start> có {count_gt} trái cây <end>"),
        
        # Câu hỏi về loại trái cây
        ("Ảnh chứa những loại trái cây nào?", f"<start> ảnh chứa {fruit_type_gt} <end>"),
        ("Trái cây trong ảnh là gì?", f"<start> trái cây trong ảnh là {fruit_type_gt} <end>"),
        ("Loại trái cây nào trong ảnh?", f"<start> {fruit_type_gt} <end>"),
        
        # Câu hỏi kết hợp
        ("Ảnh có bao nhiêu trái cây loại gì?", f"<start> ảnh có {count_gt} trái {fruit_type_gt} <end>"),
        ("Trong ảnh có bao nhiêu trái cây và là loại nào?", f"<start> trong ảnh có {count_gt} trái {fruit_type_gt} <end>")
    ]
    
    json_data = []
    for question_text, answer_text in qa_pairs:
        questions.append(question_text)
        answers.append(answer_text)
        image_features.append(feature_vector)
        image_paths_list.append(image_path)  # Lưu image path cho mỗi QA pair
        json_data.append({
            "image": image_path,  # Lưu đường dẫn đầy đủ của ảnh
            "question": question_text,
            "answer": answer_text.split(" <end>")[0].replace("<start> ", "")
        })
    
    # Lưu file JSON cho mỗi ảnh
    json_filename = os.path.join(OUTPUT_DIR, f"{filename}.json")
    with open(json_filename, "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False)

# Huấn luyện tokenizer với các câu hỏi, câu trả lời và danh sách các loại trái cây
tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(questions + answers + fruit_classes)

# Chuyển các câu hỏi và câu trả lời thành chuỗi số
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

# Padding cho các chuỗi
max_length = max(len(seq) for seq in question_sequences + answer_sequences)
question_padded = pad_sequences(question_sequences, maxlen=max_length, padding="post")
answer_padded = pad_sequences(answer_sequences, maxlen=max_length, padding="post")
print(f"Max length: {max_length}")

# Tạo thư mục lưu dữ liệu đầu ra nếu chưa tồn tại
os.makedirs("seq2seqData", exist_ok=True)
np.save("seq2seqData/question_padded.npy", question_padded.astype(np.float32))
np.save("seq2seqData/answer_padded.npy", answer_padded.astype(np.float32))
np.save("seq2seqData/image_features.npy", np.array(image_features, dtype=np.float32))
np.save("seq2seqData/image_paths.npy", np.array(image_paths_list))

with open("seq2seqData/tokenizer.json", "w", encoding="utf-8") as f:
    f.write(tokenizer.to_json())

print("Dataset mới với nhiều loại câu hỏi đã được tạo!")


Max length: 11
Dataset mới với nhiều loại câu hỏi đã được tạo!
